In [1]:
import pandas as pd
import numpy as np
import pylab as plt
%matplotlib inline

## Open up GPA data

In [11]:
gpa_data_raw = pd.read_csv("data/FR_GPA_by_Inst_data_converted.csv")
gpa_data_raw.head(20)

,Calculation1,Campus,City,County,Fall Term,Measure Names,School,Measure Values
0,21ST CENTURY EXPERIMENTAL SCH694223,Santa Cruz,n/a,Not Applicable,2017,Enrl GPA,21ST CENTURY EXPERIMENTAL SCH,NaN
1,21ST CENTURY EXPERIMENTAL SCH694223,Santa Barbara,n/a,Not Applicable,2017,Enrl GPA,21ST CENTURY EXPERIMENTAL SCH,NaN
2,21ST CENTURY EXPERIMENTAL SCH694223,San Diego,n/a,Not Applicable,2017,Enrl GPA,21ST CENTURY EXPERIMENTAL SCH,NaN
3,21ST CENTURY EXPERIMENTAL SCH694223,Los Angeles,n/a,Not Applicable,2017,Enrl GPA,21ST CENTURY EXPERIMENTAL SCH,NaN
4,21ST CENTURY EXPERIMENTAL SCH694223,Irvine,n/a,Not Applicable,2017,Enrl GPA,21ST CENTURY EXPERIMENTAL SCH,NaN
5,21ST CENTURY EXPERIMENTAL SCH694223,Davis,n/a,Not Applicable,2017,Enrl GPA,21ST CENTURY EXPERIMENTAL SCH,NaN
6,21ST CENTURY EXPERIMENTAL SCH694223,Berkeley,n/a,Not Applicable,2017,Enrl GPA,21ST CENTURY EXPERIMENTAL SCH,NaN
7,21ST CENTURY EXPERIMENTAL SCH694223,Universitywide,n/a,Not Applicable,2017,Enrl GPA,21ST CENTURY EXPERIMENTAL SCH,3.986667
8,21ST CENTURY EXPERIMENTAL SCH694223,Santa Cruz,n/a,Not Applicable,2017,Adm GPA,21ST CENTURY EXPERIMENTAL SCH,NaN
9,21ST CENTURY EXPERIMENTAL SCH694223,Santa Barbara,n/a,Not Applicable,2017,Adm GPA,21ST CENTURY EXPERIMENTAL SCH,3.999091


In [13]:
gpa_data = gpa_data_raw.drop(columns="School")
renaming = {"Uad Uc Ethn 6 Cat": "Ethnicity", 
             "Calculation1": "school",
             "Campus": "campus",
             "City": "city",
             "County": "region",
             "Fall Term":"year",
             "Measure Names": "type",
             'Uad Uc Ethn 6 Cat':"ethnicity",
             "Measure Values": "num"}
gpa_data = gpa_data.rename(index=str, columns=renaming)
gpa_data.head(10)

,school,campus,city,region,year,type,num
0,21ST CENTURY EXPERIMENTAL SCH694223,Santa Cruz,n/a,Not Applicable,2017,Enrl GPA,NaN
1,21ST CENTURY EXPERIMENTAL SCH694223,Santa Barbara,n/a,Not Applicable,2017,Enrl GPA,NaN
2,21ST CENTURY EXPERIMENTAL SCH694223,San Diego,n/a,Not Applicable,2017,Enrl GPA,NaN
3,21ST CENTURY EXPERIMENTAL SCH694223,Los Angeles,n/a,Not Applicable,2017,Enrl GPA,NaN
4,21ST CENTURY EXPERIMENTAL SCH694223,Irvine,n/a,Not Applicable,2017,Enrl GPA,NaN
5,21ST CENTURY EXPERIMENTAL SCH694223,Davis,n/a,Not Applicable,2017,Enrl GPA,NaN
6,21ST CENTURY EXPERIMENTAL SCH694223,Berkeley,n/a,Not Applicable,2017,Enrl GPA,NaN
7,21ST CENTURY EXPERIMENTAL SCH694223,Universitywide,n/a,Not Applicable,2017,Enrl GPA,3.986667
8,21ST CENTURY EXPERIMENTAL SCH694223,Santa Cruz,n/a,Not Applicable,2017,Adm GPA,NaN
9,21ST CENTURY EXPERIMENTAL SCH694223,Santa Barbara,n/a,Not Applicable,2017,Adm GPA,3.999091


In [ ]:
variables = [gpa_data["school"].unique(), gpa_data["year"], gpa_data["campus"].unique()]
# mi = pd.MultiIndex.from_product(variables, names=['school', 'year', 'campus'])
# mi

## Open up count data

In [3]:
hs_data_raw = pd.read_csv("data/HS_by_Year_data_converted.csv")
print(hs_data_raw.columns)
hs_data_raw.head()

Index(['Calculation1', 'Campus', 'City', 'County/State/ Territory',
       'Fall Term', 'Measure Names', 'Uad Uc Ethn 6 Cat', 'Measure Values'],
      dtype='object')


,Calculation1,Campus,City,County/State/ Territory,Fall Term,Measure Names,Uad Uc Ethn 6 Cat,Measure Values
0,21ST CENTURY EXPERIMENTAL SCH694223,Universitywide,NaN,"CHINA, PEOPLES REPUBLIC",2017,enr,All,12.0
1,21ST CENTURY EXPERIMENTAL SCH694223,Universitywide,NaN,"CHINA, PEOPLES REPUBLIC",2017,adm,All,30.0
2,21ST CENTURY EXPERIMENTAL SCH694223,Universitywide,NaN,"CHINA, PEOPLES REPUBLIC",2017,app,All,43.0
3,21ST CENTURY EXPERIMENTAL SCH694223,Universitywide,NaN,"CHINA, PEOPLES REPUBLIC",2016,enr,Inter- national,5.0
4,21ST CENTURY EXPERIMENTAL SCH694223,Universitywide,NaN,"CHINA, PEOPLES REPUBLIC",2016,adm,Inter- national,12.0


In [14]:
# hs_data = hs_data_raw.drop(columns="School")
renaming = {"Uad Uc Ethn 6 Cat": "Ethnicity", 
             "Calculation1": "school",
             "Campus": "campus",
             "City": "city",
             "County/State/ Territory": "region",
             "Fall Term":"year",
             "Measure Names": "type",
             'Uad Uc Ethn 6 Cat':"ethnicity",
             "Measure Values": "num"}
hs_data = hs_data_raw.rename(index=str, columns=renaming)
hs_data

,school,campus,city,region,year,type,ethnicity,num
0,21ST CENTURY EXPERIMENTAL SCH694223,Universitywide,NaN,"CHINA, PEOPLES REPUBLIC",2017,enr,All,12.0
1,21ST CENTURY EXPERIMENTAL SCH694223,Universitywide,NaN,"CHINA, PEOPLES REPUBLIC",2017,adm,All,30.0
2,21ST CENTURY EXPERIMENTAL SCH694223,Universitywide,NaN,"CHINA, PEOPLES REPUBLIC",2017,app,All,43.0
3,21ST CENTURY EXPERIMENTAL SCH694223,Universitywide,NaN,"CHINA, PEOPLES REPUBLIC",2016,enr,Inter- national,5.0
4,21ST CENTURY EXPERIMENTAL SCH694223,Universitywide,NaN,"CHINA, PEOPLES REPUBLIC",2016,adm,Inter- national,12.0
5,21ST CENTURY EXPERIMENTAL SCH694223,Universitywide,NaN,"CHINA, PEOPLES REPUBLIC",2016,app,Inter- national,18.0
6,21ST CENTURY EXPERIMENTAL SCH694223,Universitywide,NaN,"CHINA, PEOPLES REPUBLIC",2016,enr,All,5.0
7,21ST CENTURY EXPERIMENTAL SCH694223,Universitywide,NaN,"CHINA, PEOPLES REPUBLIC",2016,adm,All,12.0
8,21ST CENTURY EXPERIMENTAL SCH694223,Universitywide,NaN,"CHINA, PEOPLES REPUBLIC",2016,app,All,19.0
9,21ST CENTURY EXPERIMENTAL SCH694223,Universitywide,NaN,"CHINA, PEOPLES REPUBLIC",2015,enr,All,3.0


In [ ]:
hs_data["school"].nunique()